In [1]:
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K
from keras.utils import to_categorical

num_classes = 10

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
random.seed(42)
np.random.seed(42)

# Needed routines

In [3]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)


def create_pairs_multitask(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    class_1, class_2 = [], []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]] # created same class
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes # took next
            z1, z2 = digit_indices[d][i], digit_indices[dn][i] 
            pairs += [[x[z1], x[z2]]] # created different classes
            labels += [1, 0] # added "same" and "different"
            class_1 += [d, d]
            class_2 += [d, dn]
    return np.array(pairs), np.array(labels), np.array(class_1), np.array(class_2)

In [4]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

# Loading data

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y, tr_c1, tr_c2 = create_pairs_multitask(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y, te_c1, te_c2 = create_pairs_multitask(x_test, digit_indices)

tr_c1, tr_c2 = to_categorical(tr_c1), to_categorical(tr_c2)
te_c1, te_c2 = to_categorical(te_c1), to_categorical(te_c2)

# Defining network

In [6]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 64)           50240       input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]                    
          

# Training

In [8]:
# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=10,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

Train on 108400 samples, validate on 17820 samples
Epoch 1/10
108400/108400 [==============================] - 4s 41us/step - loss: 0.1922 - val_loss: 0.1185
Epoch 2/10
108400/108400 [==============================] - 4s 38us/step - loss: 0.1597 - val_loss: 0.1181
Epoch 3/10
108400/108400 [==============================] - 4s 37us/step - loss: 0.1551 - val_loss: 0.1128
Epoch 4/10
108400/108400 [==============================] - 4s 39us/step - loss: 0.1541 - val_loss: 0.1166
Epoch 5/10
108400/108400 [==============================] - 4s 36us/step - loss: 0.1512 - val_loss: 0.1126
Epoch 6/10
108400/108400 [==============================] - 4s 38us/step - loss: 0.1521 - val_loss: 0.1131
Epoch 7/10
108400/108400 [==============================] - 4s 39us/step - loss: 0.1507 - val_loss: 0.1169
Epoch 8/10
108400/108400 [==============================] - 5s 42us/step - loss: 0.1505 - val_loss: 0.1137
Epoch 9/10
108400/108400 [==============================] - 5s 43us/step - loss: 0.1509 - val

In [9]:
# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 85.47%
* Accuracy on test set: 85.61%


# Defining multitask network

In [10]:
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

class_1_out = Dense(num_classes, activation = 'softmax')(processed_a)
class_2_out = Dense(num_classes, activation = 'softmax')(processed_b)

model = Model([input_a, input_b], [distance, class_1_out, class_2_out])

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
model_3 (Model)                 (None, 64)           50240       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1)            0           model_3[1][0]                    
          

# Training

In [12]:
# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], [tr_y, tr_c1, tr_c2],
          batch_size=128,
          epochs=10,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], [te_y, te_c1, te_c2]))

Train on 108400 samples, validate on 17820 samples
Epoch 1/10
108400/108400 [==============================] - 6s 52us/step - loss: 1.6357 - lambda_2_loss: 0.1835 - dense_3_loss: 0.7263 - dense_4_loss: 0.7259 - val_loss: 1.5624 - val_lambda_2_loss: 0.1175 - val_dense_3_loss: 0.7225 - val_dense_4_loss: 0.7224
Epoch 2/10
108400/108400 [==============================] - 5s 48us/step - loss: 1.5919 - lambda_2_loss: 0.1507 - dense_3_loss: 0.7206 - dense_4_loss: 0.7207 - val_loss: 1.5435 - val_lambda_2_loss: 0.1068 - val_dense_3_loss: 0.7183 - val_dense_4_loss: 0.7184
Epoch 3/10
108400/108400 [==============================] - 5s 50us/step - loss: 1.5834 - lambda_2_loss: 0.1466 - dense_3_loss: 0.7184 - dense_4_loss: 0.7184 - val_loss: 1.5377 - val_lambda_2_loss: 0.1044 - val_dense_3_loss: 0.7166 - val_dense_4_loss: 0.7167
Epoch 4/10
108400/108400 [==============================] - 5s 47us/step - loss: 1.5793 - lambda_2_loss: 0.1444 - dense_3_loss: 0.7174 - dense_4_loss: 0.7175 - val_loss: 1.

In [13]:
# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])[0]
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])[0]
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 88.49%
* Accuracy on test set: 88.54%
